In [2]:
import os
import pandas as pd
from pathlib import Path
from typing import Iterator

from docling.document_converter import DocumentConverter
from langchain_core.document_loaders import BaseLoader
from langchain_core.documents import Document as LCDocument
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_ibm import WatsonxEmbeddings
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames as EmbedParams

from dotenv import load_dotenv
load_dotenv()

# Définition des constantes pour les variables d'environnement
WATSONX_APIKEY = os.getenv("WATSONX_APIKEY", "")
WATSONX_PROJECT_ID = os.getenv("PROJECT_ID", "")
WATSONX_URL = os.getenv("WATSONX_URL", "https://us-south.ml.cloud.ibm.com/")

os.environ["WATSONX_URL"] = WATSONX_URL
os.environ["WATSONX_APIKEY"] = WATSONX_APIKEY
os.environ["WATSONX_PROJECT_ID"] = WATSONX_PROJECT_ID

class DoclingPDFLoader(BaseLoader):
    def __init__(self, file_path: str | list[str]) -> None:
        self._file_paths = file_path if isinstance(file_path, list) else [file_path]
        self._converter = DocumentConverter()

    def lazy_load(self) -> Iterator[LCDocument]:
        for source in self._file_paths:
            dl_doc = self._converter.convert(source).document
            text = dl_doc.export_to_markdown()
            yield LCDocument(page_content=text)


def create_documents(path):
    loader = DoclingPDFLoader(file_path=path)
    file_name = loader._converter.convert(path).document.name
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
    )
    docs = loader.load()
    docs[0].metadata = {"filename": file_name}
    return text_splitter.split_documents(docs)


embed_params = {
    EmbedParams.TRUNCATE_INPUT_TOKENS: 512,
    EmbedParams.RETURN_OPTIONS: {"input_text": True},
}

embeddings = WatsonxEmbeddings(
    model_id="intfloat/multilingual-e5-large",
    url=os.getenv("WATSONX_URL", ""),
    apikey=os.getenv("WATSONX_APIKEY", ""),
    project_id=os.getenv("WATSONX_PROJECT_ID", ""),
    params=embed_params,
)


def create_vectorstore(path):
    texts = create_documents(path)
    return Chroma.from_documents(texts, embeddings)


doc_folder = "Documents RAG"

def update_doc_folder(folder):
    folder_list_dir = os.listdir(folder)
    folder_list_dir = [f for f in folder_list_dir if f != ".gitignore"]
    return folder_list_dir


def add_documents(vectorbase, path):
    new_documents = create_documents(path)
    vectorbase.add_documents(new_documents)


docsearch = None
files_name = update_doc_folder(doc_folder)

document_dataframe = []

if files_name:
    file_path = os.path.join(doc_folder, files_name[0])
    docsearch = create_vectorstore(file_path)

for filename in files_name:
    if docsearch:
        if filename not in [doc["filename"] for doc in docsearch.get()["metadatas"] if doc]:
            add_documents(docsearch, os.path.join(doc_folder, filename))

document_dataframe = list(set([doc["filename"] for doc in docsearch.get()["metadatas"] if doc]))

pd.DataFrame({"Documents": document_dataframe}).to_csv("document_list.csv", index=False)


In [10]:
from crewai import Agent, Task, Crew
from crewai_tools import PandasDataAnalysisTool, DocumentAnalysisTool, OpenAITool, APITool
import pandas as pd

# Tools Definition
pandas_tool = PandasDataAnalysisTool()
doc_tool = DocumentAnalysisTool()
ai_tool = OpenAITool()
api_tool = APITool()

# Agents Definition

## DataCore Analyst
DataCore = Agent(
    name="DataCore Analyst",
    backstory="A data scientist with expertise in statistical modeling and business intelligence.",
    goal="Transform raw Planning Analytics data into clean, structured information ready for AI insights.",
    tools=[pandas_tool],
    verbose=True
)

## DocuMentor Analyst
DocuMentor = Agent(
    name="DocuMentor Analyst",
    backstory="An NLP expert skilled in extracting insights from internal business documents.",
    goal="Analyze internal documents to identify trends and contextual insights.",
    tools=[doc_tool],
    verbose=True
)

## Insight Synthesizer
InsightSynthesizer = Agent(
    name="Insight Synthesizer",
    backstory="A strategist blending AI-driven analytics with business insights.",
    goal="Merge quantitative trends and qualitative insights into actionable business recommendations.",
    tools=[ai_tool],
    verbose=True
)

## Strategy Navigator
StrategyNavigator = Agent(
    name="Strategy Navigator",
    backstory="A business strategist ensuring insights align with company goals and market trends.",
    goal="Validate insights, prioritize actions, and align findings with business strategy.",
    tools=[ai_tool],
    verbose=True
)

## Tech Integrator
TechIntegrator = Agent(
    name="Tech Integrator",
    backstory="A systems engineer ensuring seamless integration of AI and analytics into workflows.",
    goal="Automate workflows, connect IBM Planning Analytics with AI models, and monitor performance.",
    tools=[api_tool],
    verbose=True
)

# Task Definitions

data_task = Task(
    description="Clean and analyze Planning Analytics data to extract trends and anomalies.",
    agent=DataCore
)

doc_task = Task(
    description="Analyze internal documents and extract relevant business insights.",
    agent=DocuMentor
)

insight_task = Task(
    description="Synthesize quantitative and qualitative insights into actionable recommendations.",
    agent=InsightSynthesizer
)

strategy_task = Task(
    description="Validate insights and prioritize strategic actions for implementation.",
    agent=StrategyNavigator
)

tech_task = Task(
    description="Ensure seamless technical integration between AI models and Planning Analytics data.",
    agent=TechIntegrator
)

# Crew Assembly
crew = Crew(
    agents=[DataCore, DocuMentor, InsightSynthesizer, StrategyNavigator, TechIntegrator],
    tasks=[data_task, doc_task, insight_task, strategy_task, tech_task],
    verbose=True
)

# Run the Crew
crew.kickoff()


d:\Applications\Tm1\Tango_Core_Model\Data\Python_Scripts\PAAgenticAnalysis\.venv\Lib\site-packages\pydantic\_internal\_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)


ImportError: cannot import name 'PandasDataAnalysisTool' from 'crewai_tools' (d:\Applications\Tm1\Tango_Core_Model\Data\Python_Scripts\PAAgenticAnalysis\.venv\Lib\site-packages\crewai_tools\__init__.py)

In [17]:
from TM1py.Services import TM1Service
from TM1py.Utils.Utils import (
    build_pandas_dataframe_from_cellset,
    build_cellset_from_pandas_dataframe,
)
import configparser

config = configparser.ConfigParser()
config.read("config.ini")
cube_name="00.Ventes"
view_name="01.Analyse Couts (4 pays)"
with TM1Service(**config["tango_core_model"]) as tm1:

        df = tm1.cells.execute_view_dataframe(
            cube_name="00.Ventes",view_name="01.Analyse Couts (4 pays)"
        ) 
        measure_dim = tm1.cubes.get_measure_dimension(cube_name=cube_name)
        country_dim = "Pays"
        period_dim = "Period"

        # vérification d'existence d'alias pour la dimension indicateur (autre que l'attribut format)
        measure_alias_names = tm1.elements.get_element_attribute_names(
            measure_dim, measure_dim
        )
        period_alias_names = tm1.elements.get_element_attribute_names(
            period_dim, period_dim
        )

        period_alias = tm1.elements.get_attribute_of_elements(
            period_dim, period_dim, "French"
        )

        def round_2(number):
            """
            Converts str to float and round to the tenth
            """
            try:
                return round(float(number), 1)
            except:
                print("not a str")

        def rename_period_alias(df):
            """
            Renames the periods column with period alias
            """
            df.rename(columns=period_alias, inplace=True)

        def preprocessing(df):
            """
            Preprocessing the dataframe
            """
            df = df.fillna(0)
            rename_period_alias(df)
            for col in df.columns[-12:]:
                df[col] = df[col].apply(round_2)
            # df = df.set_index(df.columns[0])
            return df

        def dimension_of_element(cube_name, view_name, element):
            cellset_sample = list(
                tm1.cubes.cells.execute_view(
                    cube_name=cube_name, view_name=view_name
                ).keys()
            )[0]
            for dim in cellset_sample:
                if element in dim:
                    first_bracket_index = dim.index("[")
                    second_bracket_index = dim.index("]")
                    return dim[first_bracket_index + 1 : second_bracket_index]

        def get_context(cube_name, view_name):
            list_context = tm1.cubes.cells.execute_view_ui_dygraph(
                cube_name=cube_name, view_name=view_name, skip_zeros=False
            )["titles"][0]["name"]
            list_context = list_context.split(" / ")
            context = "For the following dataframe, the "
            for i, element in enumerate(list_context):
                elem_dim = dimension_of_element(cube_name, view_name, element)

                alias_name = tm1.elements.get_alias_element_attributes(
                    dimension_name=elem_dim, hierarchy_name=elem_dim
                )[-1]
                try:
                    elem_alias = list(
                        tm1.elements.get_attribute_of_elements(
                            dimension_name=elem_dim,
                            hierarchy_name=elem_dim,
                            elements=[element],
                            attribute=alias_name,
                        ).values()
                    )[0]
                except:
                    elem_alias = element
                context += f"{elem_dim} is {elem_alias}"
                if i < len(list_context) - 1:
                    context += " and the "
            context = context.replace(measure_dim, "data displayed")
            return context

        def view_dataframe(cube_name, view_name):
            return preprocessing(
                tm1.cubes.cells.execute_view_dataframe_shaped(
                    cube_name=cube_name, view_name=view_name, skip_zeros=False
                )
            )

        df = view_dataframe(cube_name, view_name)

d:\Applications\Tm1\Tango_Core_Model\Data\Python_Scripts\PAAgenticAnalysis\.venv\Lib\site-packages\TM1py\Services\TM1Service.py:90: DeprecationWarning: Server Service will be moved to a new location in a future version
  self.server = ServerService(self._tm1_rest)
d:\Applications\Tm1\Tango_Core_Model\Data\Python_Scripts\PAAgenticAnalysis\.venv\Lib\site-packages\TM1py\Services\TM1Service.py:91: DeprecationWarning: Monitoring Service will be moved to a new location in a future version
  self.monitoring = MonitoringService(self._tm1_rest)


In [18]:
df

,Pays,Indicateurs_Activité,janv-24,févr-24,mars-24,avr-24,mai-24,juin-24,juil-24,août-24,sept-24,oct-24,nov-24,déc-24
0,Finlande,Coûts Commerciaux,671.8,64.5,583.4,519.2,122.6,309.8,679.2,208.9,682.4,481.9,263.7,39.2
1,Finlande,Coûts généraux,257.1,424.2,473.9,114.7,553.2,466.8,61.3,522.4,489.7,181.4,58.7,421.9
2,Finlande,Coûts Logistiques,692.4,521.6,352.8,321.9,702.5,118.7,171.3,183.9,118.2,699.4,374.1,402.3
3,Finlande,Coûts Stockage,618.9,91.3,423.5,229.2,83.7,121.5,38.4,598.3,153.2,88.9,673.4,54.8
4,Finlande,Coûts Marketing,263.5,109.8,184.7,313.2,624.1,531.7,318.9,512.6,401.3,517.2,398.5,472.4
5,Finlande,Coûts Maintenance,158.7,664.1,436.8,462.3,306.5,271.8,633.9,378.5,384.2,219.4,506.2,369.8
6,Suède,Coûts Commerciaux,655.9,69.7,569.4,507.1,129.4,318.3,663.4,214.6,664.8,469.2,272.1,43.7
7,Suède,Coûts généraux,264.2,412.9,459.8,120.3,540.9,453.7,67.1,508.4,476.1,189.2,62.5,410.8
8,Suède,Coûts Logistiques,673.5,509.3,341.2,310.8,681.4,125.6,177.4,192.8,124.3,679.6,362.5,390.7
9,Suède,Coûts Stockage,601.8,94.2,414.7,236.1,88.1,126.4,42.9,579.2,158.7,91.3,654.1,57.4


In [37]:
df.drop('Indicateurs_Activité',axis=1).groupby(['Pays']).sum()#.drop('Indicateurs_Activité',axis=1).sum(axis=1)

,janv-24,févr-24,mars-24,avr-24,mai-24,juin-24,juil-24,août-24,sept-24,oct-24,nov-24,déc-24
Pays,,,,,,,,,,,,
Belgique,2018.3,1947.9,3347.9,1363.4,2498.4,2823.4,1708.0,2743.4,1958.2,2163.4,1713.2,2368.4
Finlande,2662.4,1875.5,2455.1,1960.5,2392.6,1820.3,1903.0,2404.6,2229.0,2188.2,2274.6,1760.4
Portugal,2035.9,1962.4,3359.2,1387.4,2526.0,2851.2,1725.4,2763.1,1975.4,2180.2,1728.5,2388.0
Suède,2633.9,1843.6,2400.3,1932.5,2365.4,1823.9,1894.4,2382.8,2207.6,2160.3,2253.2,1723.5


In [26]:
result = df[['janv-24', 'févr-24', 'mars-24', 'avr-24', 'mai-24', 'juin-24', 'juil-24', 'août-24', 'sept-24', 'oct-24', 'nov-24', 'déc-24']].pct_change(axis=1)#.dropna(axis=1, how='all')
result

,janv-24,févr-24,mars-24,avr-24,mai-24,juin-24,juil-24,août-24,sept-24,oct-24,nov-24,déc-24
0,NaN,-0.903989,8.044961,-0.110045,-0.763867,1.526917,1.192382,-0.692432,2.266635,-0.293816,-0.452791,-0.851346
1,NaN,0.649942,0.117162,-0.757966,3.823017,-0.156182,-0.868680,7.522023,-0.062596,-0.629569,-0.676406,6.187394
2,NaN,-0.246678,-0.323620,-0.087585,1.182355,-0.831032,0.443134,0.073555,-0.357259,4.917090,-0.465113,0.075381
3,NaN,-0.852480,3.638554,-0.458796,-0.634817,0.451613,-0.683951,14.580729,-0.743941,-0.419713,6.574803,-0.918622
4,NaN,-0.583302,0.682149,0.695723,0.992656,-0.148053,-0.400226,0.607400,-0.217128,0.288811,-0.229505,0.185445
5,NaN,3.184625,-0.342268,0.058379,-0.337011,-0.113214,1.332230,-0.402903,0.015059,-0.428943,1.307201,-0.269459
6,NaN,-0.893734,7.169297,-0.109413,-0.744824,1.459815,1.084197,-0.676515,2.097856,-0.294224,-0.420077,-0.839397
7,NaN,0.562831,0.113587,-0.738365,3.496259,-0.161213,-0.852105,6.576751,-0.063533,-0.602604,-0.669662,5.572800
8,NaN,-0.243801,-0.330061,-0.089097,1.192407,-0.815674,0.412420,0.086809,-0.355290,4.467418,-0.466598,0.077793
9,NaN,-0.843470,3.402335,-0.430673,-0.626853,0.434733,-0.660601,12.501166,-0.726001,-0.424701,6.164294,-0.912246
